<a href="https://colab.research.google.com/github/pko89403/ScalaAndSpark/blob/master/RapidsAI_XGBoost_On_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jul 10 13:38:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.1/cudf-0.9.1-cuda10-1.jar

--2020-07-10 13:38:37--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.52.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.52.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64759406 (62M) [application/java-archive]
Saving to: ‘cudf-0.9.2-cuda10-1.jar’

cudf-0.9.2-cuda10-1 100%[===================>]  61.76M   161MB/s    in 0.4s    

2020-07-10 13:38:40 (161 MB/s) - ‘cudf-0.9.2-cuda10-1.jar’ saved [64759406/64759406]



In [3]:
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar

--2020-07-10 13:38:41--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.52.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.52.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349225621 (333M) [application/java-archive]
Saving to: ‘xgboost4j_2.x-1.0.0-Beta5.jar’

xgboost4j_2.x-1.0.0 100%[===================>] 333.05M   244MB/s    in 1.4s    

2020-07-10 13:38:48 (244 MB/s) - ‘xgboost4j_2.x-1.0.0-Beta5.jar’ saved [349225621/349225621]



In [4]:
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar

--2020-07-10 13:38:50--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.52.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.52.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4829218 (4.6M) [application/java-archive]
Saving to: ‘xgboost4j-spark_2.x-1.0.0-Beta5.jar’

xgboost4j-spark_2.x 100%[===================>]   4.61M  --.-KB/s    in 0.1s    

2020-07-10 13:38:50 (33.5 MB/s) - ‘xgboost4j-spark_2.x-1.0.0-Beta5.jar’ saved [4829218/4829218]



In [5]:
!apt-get install openjdk-8-jdk-headless

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 33 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u252-b09-1~18.04 [27.5 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless amd64 8u252-b09-1~18.04 [8,250 kB]


In [6]:
!wget http://mirror.navercorp.com/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

--2020-07-10 13:39:07--  http://mirror.navercorp.com/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving mirror.navercorp.com (mirror.navercorp.com)... 125.209.216.167
Connecting to mirror.navercorp.com (mirror.navercorp.com)|125.209.216.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232530699 (222M) [application/octet-stream]
Saving to: ‘spark-2.4.5-bin-hadoop2.7.tgz’

spark-2.4.5-bin-had 100%[===================>] 221.76M  2.13MB/s    in 23s     

2020-07-10 13:39:30 (9.84 MB/s) - ‘spark-2.4.5-bin-hadoop2.7.tgz’ saved [232530699/232530699]



In [7]:
!apt-get install tar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tar is already the newest version (1.29b-2ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [8]:
!tar -xf spark-2.4.5-bin-hadoop2.7.tgz

In [9]:
!ls

cudf-0.9.2-cuda10-1.jar    spark-2.4.5-bin-hadoop2.7.tgz
drive			   xgboost4j_2.x-1.0.0-Beta5.jar
sample_data		   xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.5-bin-hadoop2.7


In [10]:
!pip install pyspark==2.4.5 findspark

     |████████████████████████████████| 217.8MB 64kB/s 
     |████████████████████████████████| 204kB 46.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=92af766539e0e59e7c84f448029cd8980a27a527356fc7be330450f38b6180ab
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [11]:
!wget https://github.com/rapidsai/spark-examples/raw/master/datasets/mortgage-small.tar.gz
!tar xvf mortgage-small.tar.gz

--2020-07-10 13:40:20--  https://github.com/rapidsai/spark-examples/raw/master/datasets/mortgage-small.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/spark-examples/master/datasets/mortgage-small.tar.gz [following]
--2020-07-10 13:40:20--  https://raw.githubusercontent.com/rapidsai/spark-examples/master/datasets/mortgage-small.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103288 (101K) [application/octet-stream]
Saving to: ‘mortgage-small.tar.gz’

mortgage-small.tar. 100%[===================>] 100.87K  --.-KB/s    in 0.02s   

2020-07-10 13:40:20 (4.27 MB/s) - ‘mortgage-smal

In [12]:
!pwd

/content


In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME']="/content/spark-2.4.5-bin-hadoop2.7"

In [14]:
try:
    spark.stop
except:
    pass

In [15]:
import findspark
import pyspark
from pyspark.sql.session import SparkSession

findspark.init()


spark = (SparkSession
    .builder
    .appName("myspark")
    .config('spark.jars', '/content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar')         
    .getOrCreate())

spark.sparkContext.addPyFile("/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar")



In [16]:
print(spark.version)

2.4.5


In [17]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time

### Data reader uses GpuDataReader(spark) 
instead of spark.read 2. setFeaturesCol function on CPU is replaced by setFeaturesCols(features) a. 
> This also eliminates the need for vectorization (assembling multiple feature columns in to one column) since we can read multiple columns

In [18]:
label = 'delinquency_12'
schema = StructType([
    StructField('orig_channel', FloatType()),
    StructField('first_home_buyer', FloatType()),
    StructField('loan_purpose', FloatType()),
    StructField('property_type', FloatType()),
    StructField('occupancy_status', FloatType()),
    StructField('property_state', FloatType()),
    StructField('product_type', FloatType()),
    StructField('relocation_mortgage_indicator', FloatType()),
    StructField('seller_name', FloatType()),
    StructField('mod_flag', FloatType()),
    StructField('orig_interest_rate', FloatType()),
    StructField('orig_upb', IntegerType()),
    StructField('orig_loan_term', IntegerType()),
    StructField('orig_ltv', FloatType()),
    StructField('orig_cltv', FloatType()),
    StructField('num_borrowers', FloatType()),
    StructField('dti', FloatType()),
    StructField('borrower_credit_score', FloatType()),
    StructField('num_units', IntegerType()),
    StructField('zip', IntegerType()),
    StructField('mortgage_insurance_percent', FloatType()),
    StructField('current_loan_delinquency_status', IntegerType()),
    StructField('current_actual_upb', FloatType()),
    StructField('interest_rate', FloatType()),
    StructField('loan_age', FloatType()),
    StructField('msa', FloatType()),
    StructField('non_interest_bearing_upb', FloatType()),
    StructField(label, IntegerType()),
])
features = [ x.name for x in schema if x.name != label ]

train_data = GpuDataReader(spark).schema(schema).option('header', True).csv('/content/mortgage-small/train')
eval_data = GpuDataReader(spark).schema(schema).option('header', True).csv('/content/mortgage-small/eval')

In [19]:

""" For CPU
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col(label)))

# CPU version : CPU 버전은 Vectorization이 필요하다.  모든 컬럼들을 feature 컬럼 하나로 뭉쳐야한다
train_data = spark.read.schema(schema).option('header', True).csv('/home/mortgage-small/train')
eval_data = spark.read.schema(schema).option('header', True).csv('/home/mortgage-small/eval')

train_data = vectorize(train_data)
eval_data = vectorize(eval_data)
"""

" For CPU\nfrom pyspark.ml.feature import VectorAssembler\nfrom pyspark.sql.functions import col\n\ndef vectorize(data_frame):\n    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]\n    return (VectorAssembler()\n        .setInputCols(features)\n        .setOutputCol('features')\n        .transform(data_frame.select(to_floats))\n        .select(col('features'), col(label)))\n\n# CPU version : CPU 버전은 Vectorization이 필요하다.  모든 컬럼들을 feature 컬럼 하나로 뭉쳐야한다\ntrain_data = spark.read.schema(schema).option('header', True).csv('/home/mortgage-small/train')\neval_data = spark.read.schema(schema).option('header', True).csv('/home/mortgage-small/eval')\n\ntrain_data = vectorize(train_data)\neval_data = vectorize(eval_data)\n"

In [22]:
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 10, 
    'maxLeaves': 256,
    'growPolicy': 'depthwise',
    'objective': 'binary:logistic',
    'minChildWeight': 30.0,
    'lambda_': 1.0,
    'scalePosWeight': 2.0,
    'subsample': 1.0,
    'nthread': 1,
    'numRound': 100,
    'numWorkers': 1,
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
# Then build the evaluator and the hyperparameters
evaluator = (MulticlassClassificationEvaluator()
    .setLabelCol(label))
param_grid = (ParamGridBuilder()
    .addGrid(classifier.maxDepth, [3, 6])
    .addGrid(classifier.numRound, [10, 20])
    .build())
# Finally the corss validator
cross_validator = (CrossValidator()
    .setEstimator(classifier)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(param_grid)
    .setNumFolds(3))

In [23]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Cross-Validation', lambda: cross_validator.fit(train_data))

Cross-Validation takes 27.48 seconds


In [24]:
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 10, 
    'maxLeaves': 256,
    'growPolicy': 'depthwise',
    'objective': 'binary:logistic',
    'minChildWeight': 30.0,
    'lambda_': 1.0,
    'scalePosWeight': 2.0,
    'subsample': 1.0,
    'nthread': 1,
    'numRound': 100,
    'numWorkers': 1
}
# numberWorkers = Shuld be set to number of GPU Core or CPU Core  
# For GPU
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
# For CPU
#classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCol('features')

In [25]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Training', lambda: classifier.fit(train_data))

Training takes 1.04 seconds


In [28]:
def transform():
    result = model.transform(eval_data).cache()
    result.foreachPartition(lambda _: None)
    return result
result = with_benchmark('Transforming', transform)
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

Transforming takes 1.49 seconds
+--------------+--------------------+--------------------+----------+
|delinquency_12|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+----------+
|           0.0|[5.90152835845947...|[0.99727220018394...|       0.0|
|           0.0|[5.90152835845947...|[0.99727220018394...|       0.0|
|           0.0|[5.90152835845947...|[0.99727220018394...|       0.0|
|           0.0|[5.34092235565185...|[0.99523139884695...|       0.0|
|           0.0|[5.34092235565185...|[0.99523139884695...|       0.0|
+--------------+--------------------+--------------------+----------+
only showing top 5 rows



In [29]:
accuracy = with_benchmark(
    'Evaluation',
    lambda: MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result))
print('Accuracy is ' + str(accuracy))

Evaluation takes 0.19 seconds
Accuracy is 0.995580557003928


In [30]:
spark.stop()